In [3]:
import sys
sys.path.append("..")

In [4]:
import nltk, re, string, time, os
import pandas as pd
import numpy as np
# from textClassification import textAnalysis
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn import metrics 
from sklearn import cross_validation
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn
import matplotlib.pyplot as plt
%matplotlib inline

import operator
english_stopwords = stopwords.words('english')

### Helper functions

In [5]:
# automate creation of csv for kaggle submission
def create_submission(filename, prediction):
    pd.DataFrame.to_csv(pd.DataFrame(df_test['Id']).join(pd.DataFrame(prediction)), path_or_buf=filename, header=['Id','Category'], index=False)

In [6]:
# yell outloud 'process complete'
def finished():
    os.system("say 'process complete'")

In [7]:
# automate crossvalidation and printing of results
def crossvalidation(model, train_features, real_category):
    scores = cross_validation.cross_val_score(model, train_features, real_category, cv=5)
    print("Accuracy: %0.5f (+/- %0.2f)\n" % (scores.mean(), scores.std() * 2))

## Loading Data

In [8]:
df = pd.read_csv("newtrain.csv", low_memory=False)

In [9]:
df_test = pd.read_csv("newtest.csv", low_memory=False)

In [8]:
allText = ""
for i in df.Text:
    allText += i + '.'

In [9]:
f = open('allText.txt', 'w')

In [10]:
f.write(allText)
f.close()

## Testing functions from Text Analysis assignement on Yahoo questions text data

In [11]:
path = "allText.txt"

In [12]:
analysis = textAnalysis(path)

NameError: name 'textAnalysis' is not defined

In [29]:
# tokenize sentences and words an run them through a POS tagger
sentences,tagged_sents = analysis.gen_corpus_prep()

# generate noun phrases and proper nouns
nps,nps_strings,fd_nps = analysis.chunker(sentences,tagged_sents)

# deduce synsets through lesk algorithm and calculate distance between synsets
final_list,hyponyms = analysis.lesk_comparison(nps,sentences)

# print final report
analysis.final_report(fd_nps,final_list,hyponyms)

Starting NP parsing.
Done with NP parsing.
Done with FD of NPs
Extracting nouns...
Done extracting nouns
Done running Lesk on nouns
Done creating FD on lemmas

Most frequent noun phrases:
---------------------------
.how do                 91   
br & gt                 27   
.what do                22   
my boyfriend            19   
a guy                   19   
my comput               16   
the world               15   
a girl                  14   
.whi do                 11   
.i am                   9    
.i want                 9    
my husband              9    


Most frequent lemmas from NPs and their hyponyms:
-------------------------------------------------
can             194      (containerful.n.01)
get             184      (return.n.11)
know            156      (knowing.n.01)
find            89       (act.n.02)
need            77       (psychological_feature.n.01)
want            70       (need.n.01)
people          58       (group.n.01)
someone         51       (organis

## Prepping data

In [10]:
cat_dict = dict()

In [11]:
cat_dict = {1:"Business&Finance",2:"Computers&Internet",3:"Entertainment&Music",4:"Family&Relationships",5:"Education&Reference",6:"Health",7:"Science&Mathematics"}

In [10]:
# creating feature for lengths of text and converting to pandas series
lengths = []
for i in df.Text.iteritems():
    lengths.append(len(i[1]))
lengths = pd.Series(lengths)

In [11]:
# appending lengths feature to dataframe
df['lengths'] = lengths

In [12]:
# aggregate all text
all_text = []
for i in df.Text:
    all_text.append(i)

In [18]:
# aggregate all text and tokenize it
tok_sent = []
for i in df.Text:
    tok_sent.append(nltk.word_tokenize(i))

In [19]:
# pos_tag every word
pos_sent = []
for i in tok_sent:
    pos_sent.append(nltk.pos_tag(i))

## Implementation and Testing of Strategies

### 1st Strategy: Count occurrence of every tag (17.6%)

In [12]:
# creating list with all tags from upenn_tagset
tagset = ['$', "''", '(', ')', ',', '--', '.', ':','CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']

In [21]:
# creating columns of zeroes for every possible tag
for i in tagset:
    df[i] = pd.Series(np.zeros(df.shape[0]))

In [22]:
# Tokenizing and tagging every question training set
tagged_questions = []
for i in df.Text:
    tagged_questions.append(nltk.pos_tag(nltk.word_tokenize(i)))

In [23]:
# Tokenizing and tagging every question for test set
tagged_questions_test = []
for i in df_test.Text:
    tagged_questions_test.append(nltk.pos_tag(nltk.word_tokenize(i)))

In [24]:
# This is taking WAAAY too long, expected runtime > 1 hour --use sparce vector array instead
time_check = time.time()
for n,q in enumerate(tagged_questions[:10]): # for every tagged questios from 'df.Text'...
    for tag in tagset: # take the list of possible tags...
        for w in q: # for every tagged word in the question text
            if w[1] == tag: # check if the tag matches the one the word is tagged as
                df[tag][n] += 1 # if so, increase counter and update the dataframe
    if n+1 % 10 == 0:
        print("Done with first",n)
print(round((time.time() - time_check)/60, 2),"minutes")

0.41 minutes


/Users/nicolassoldi/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
# round up all tags in one list for train data
tags_only_train = []
for q in tagged_questions:
    temp = ""
    for w in q:
        temp += w[1]
        temp += " "
    tags_only_train.append(temp)

In [27]:
# round up all tags in one list for test data
tags_only_test = []
for q in tagged_questions_test:
    temp = ""
    for w in q:
        temp += w[1]
        temp += " "
    tags_only_test.append(temp)

In [13]:
# initializing vectorizer
vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=5, max_features=433)

In [14]:
# Train sparce vector array
arr_train_feature_sparse = vec.fit_transform(pd.Series(tags_only_train))
arr_train_feature = arr_train_feature_sparse.toarray()
arr_train_feature.shape

NameError: name 'tags_only_train' is not defined

In [15]:
# Test sparce vector array
arr_test_feature_sparse = vec.fit_transform(pd.Series(tags_only_test))
arr_test_feature = arr_test_feature_sparse.toarray()
arr_test_feature.shape

NameError: name 'tags_only_test' is not defined

In [ ]:
# Training logistic regression with features in sparce array (arr_train_feature) and labels (df.Category)
logreg = LogisticRegression()
logreg_model_train = logreg.fit(arr_train_feature, df.Category)

In [34]:
# predicting against train features in sparce array
logreg_predictions_train = logreg_model_train.predict(arr_train_feature)

In [35]:
# crossvalidating
crossvalidation(logreg, arr_train_feature, df.Category)

Accuracy: 0.30913 (+/- 0.02)


In [ ]:
# automatically creates submission with whatever file name you pass as the argument
create_submission("submission_3.csv", 'some prediction set')

## Creating 'Predict Function' to test subsequent strategies

In [16]:
# function to facilitate testing of variations in CountVecotrizer parameters (min_df, max_features, etc.)
def Predict(model, vec, train_set, other_set, scoring, num_neighbors=1):
    '''
    model -  LogisticRegression, MultinomialNB, KNeighborsClassifier, svm
    '''
    # Train-data sparce vectorizing
    arr_train_feature_sparse = vec.fit_transform(train_set.Text)
    arr_train_feature = arr_train_feature_sparse.toarray()
    print("arr_train_feature.shape:", arr_train_feature.shape)

    # Other data sparce vectorizing
    arr_other_feature_sparse = vec.transform(other_set.Text)
    arr_other_feature = arr_other_feature_sparse.toarray()
    print("arr_other_feature.shape:", arr_other_feature.shape,'\n')

    # Training model with features in sparce array (arr_train_feature) and labels (df.Category)
    if model == KNeighborsClassifier:
        nb = model(n_neighbors = num_neighbors)
    else:
        nb = model
        
    nb_model_train = nb.fit(arr_train_feature, train_set.Category)    

    # predicting against dev features in sparce array
    nb_predictions_other = nb_model_train.predict(arr_other_feature)
    
    # print out crossvalidation or scoring results
    if scoring == 'cv':
        crossvalidation(nb, arr_train_feature, train_set.Category)
    elif scoring == 'score': 
        print(accuracy_score(other_set.Category, nb_predictions_other))
        
    return nb_predictions_other

### Second strategy: simple vectorizer and logistic regression (20.9%)

In [19]:
logreg = LogisticRegression()

In [22]:
logreg_predictions_test = Predict(logreg, vec, df, df, 'cv')

arr_train_feature.shape: (2698, 1000)
arr_other_feature.shape: (2698, 1000) 

Accuracy: 0.47177 (+/- 0.06)



In [195]:
# automatically creates submission with whatever file name you pass as the argument
create_submission("submission_5.csv", logreg_predictions_test)

### Third strategy: bigram, trigram and tetragram vectorizer and logistic regression (22.6%)

In [23]:
# notice the ngram_range parameter has "2, 4", so ngrams range 2 to 4
vec = CountVectorizer(ngram_range=(2, 4), token_pattern=r'\b\w+\b', min_df=5, max_features=640)

In [24]:
# Predict(model, vec, train_set, other_set, scoring, num_neighbors = 1, gamma=False)
logreg_predictions_test = Predict(logreg, vec, df, df, 'cv')

arr_train_feature.shape: (2698, 640)
arr_other_feature.shape: (2698, 640) 

Accuracy: 0.34875 (+/- 0.04)



### Fourth strategy: same vectorizer but with Naive Bayes (%)

In [25]:
nb = MultinomialNB()

In [26]:
# notice the ngram_range parameter has "2, 4", so ngrams range 2 to 4
vec = CountVectorizer(ngram_range=(2, 4), token_pattern=r'\b\w+\b', min_df=5, max_features=640)
logreg_predictions_test = Predict(nb, vec, df, df, 'cv')

arr_train_feature.shape: (2698, 640)
arr_other_feature.shape: (2698, 640) 

Accuracy: 0.34686 (+/- 0.05)



### Fifth strategy: Tf-idf vectorizer

In [27]:
tfidf_vec = TfidfVectorizer(min_df=1, max_features=1000)

In [28]:
tfid_test_predictions = Predict(nb, tfidf_vec, df, df, 'cv')

arr_train_feature.shape: (2698, 1000)
arr_other_feature.shape: (2698, 1000) 

Accuracy: 0.45546 (+/- 0.04)



In [ ]:
create_submission('submission_6.csv', tfid_test_predictions)

### Proper Nouns

**develop training, dev, testing sets**

In [29]:
random_index = np.random.permutation(df.index)
df_shuffled = df.ix[random_index]
df_shuffled.reset_index(drop=True, inplace=True)
df_shuffled[:5]

,Category,Text
0,3,when will the new episodes of justice league u...
1,3,i dreamed that i am undressed in public
2,3,a simple riddle? what goes through a door but ...
3,1,what is the genesis of the name 401k?
4,3,what is worse than biting an apple &amp; seein...


In [30]:
rows, columns = df_shuffled.shape
print("Rows:", rows)
print("Columns:", columns)

Rows: 2698
Columns: 2


**80/20 split for training/ dev**

In [31]:
train_size = round(rows*.8)
dev_size   = round(rows*.2)

In [32]:
df_train = df_shuffled.loc[:train_size]
print('train:',df_train.shape)

df_dev = df_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)
print('dev:',df_dev.shape)

train: (2159, 2)
dev: (540, 2)


##Implement voting system between models

In [33]:
def vote_winner(pair):
    dic = {}
    for i in range(len(pair)):
        num = pair[i]
        if num in dic:
            dic[num] += 1
        else:
            dic[num] = 1
    return sorted(dic.items(), key=operator.itemgetter(1), reverse=True)[0][0]
     

## counting # of "I"
**Training**

In [34]:
tok_sent_train = []
for i in df_train.Text:
    tok_sent_train.append(nltk.word_tokenize(i))
    
num_i_train = []
for i in tok_sent_train:
    count = 0
    for j in i:
        if j == "i":
            count += 1
    num_i_train.append(count)
    
# num_i = pd.Series(num_i)
# df_train['num_i'] = num_i

df_train_features = pd.DataFrame(num_i_train, columns=['num_i'])

vec_count = tfidf_vec = TfidfVectorizer(min_df=1, max_features=1000)
arr_train_feature_sparse = vec_count.fit_transform(df_train.Text)
arr_train_features = arr_train_feature_sparse.toarray()
df_train_features = df_train_features.join(pd.DataFrame(arr_train_features))

**Dev**

In [35]:
tok_sent_dev = []
for i in df_dev.Text:
    tok_sent_dev.append(nltk.word_tokenize(i))
    
num_i_dev = []
for i in tok_sent_dev:
    count = 0
    for j in i:
        if j == "i":
            count += 1
    num_i_dev.append(count)
    
# num_i_dev = pd.Series(num_i)
# df_dev['num_i'] = num_i


df_dev_features = pd.DataFrame(num_i_dev, columns=['num_i'])

In [36]:
df_dev_features.head()

,num_i
0,0
1,0
2,0
3,1
4,0


In [37]:
arr_dev_features = tfidf_vec.transform(df_dev.Text)
df_dev_features = df_dev_features.join(pd.DataFrame(arr_dev_features.toarray()))

In [38]:
df_dev_features.shape

(540, 1001)

In [39]:
nb = MultinomialNB()
nb_model = nb.fit(df_train_features, df_train.Category)
nb_predictions = nb_model.predict(df_dev_features)
nb_predictions[:10]

array([1, 4, 2, 1, 1, 1, 1, 1, 1, 2])

In [40]:
accuracy_score(df_dev.Category, nb_predictions)

0.45925925925925926

In [41]:
crossvalidation(nb_model, df_train_features, df_train.Category)

Accuracy: 0.43443 (+/- 0.03)



##TFIDF

In [42]:
tfidf_vec = TfidfVectorizer(min_df=1, max_features=1190, stop_words='english')

**test on dev set**

In [43]:
nb = MultinomialNB(alpha=0.9, class_prior=None, fit_prior=False)
svm_model = svm.SVC(C=800, class_weight=None, gamma=0.0, kernel='rbf')
#train_predictions_svm = Predict(svm_model, tfidf_vec, df_train, df_train, 'cv')
logreg = LogisticRegression()
knn = KNeighborsClassifier(num_neighbors=3)

In [44]:
dev_predictions_nb = Predict(nb, tfidf_vec, df_train, df_dev, 'score')
dev_predictions_logreg = Predict(logreg, tfidf_vec, df_train, df_dev, 'score')
dev_predictions_KNN = Predict(KNeighborsClassifier, tfidf_vec, df_train, df_dev, 'score')
dev_predictions_svm = Predict(svm_model, tfidf_vec, df_train, df_dev, 'cv')

arr_train_feature.shape: (2159, 1190)
arr_other_feature.shape: (540, 1190) 

0.542592592593
arr_train_feature.shape: (2159, 1190)
arr_other_feature.shape: (540, 1190) 

0.535185185185
arr_train_feature.shape: (2159, 1190)
arr_other_feature.shape: (540, 1190) 

0.331481481481
arr_train_feature.shape: (2159, 1190)
arr_other_feature.shape: (540, 1190) 

Accuracy: 0.49744 (+/- 0.02)



In [45]:
voted_results_dev = [vote_winner(pair) for pair in zip(dev_predictions_logreg, dev_predictions_nb, dev_predictions_svm)]

In [48]:
train_predictions_nb = Predict(nb, tfidf_vec, df, df, 'cv')
train_predictions_logreg = Predict(logreg, tfidf_vec, df, df, 'cv')
train_predictions_KNN = Predict(KNeighborsClassifier, tfidf_vec, df, df, 'cv')
train_predictions_svm = Predict(svm_model, tfidf_vec, df, df, 'cv')

arr_train_feature.shape: (2698, 1190)
arr_other_feature.shape: (2698, 1190) 

Accuracy: 0.53256 (+/- 0.03)

arr_train_feature.shape: (2698, 1190)
arr_other_feature.shape: (2698, 1190) 

Accuracy: 0.50068 (+/- 0.04)

arr_train_feature.shape: (2698, 1190)
arr_other_feature.shape: (2698, 1190) 

Accuracy: 0.31615 (+/- 0.05)

arr_train_feature.shape: (2698, 1190)
arr_other_feature.shape: (2698, 1190) 

Accuracy: 0.52325 (+/- 0.06)



In [53]:

print(df.Category.shape,dev_predictions_logreg.shape,dev_predictions_nb.shape,dev_predictions_svm.shape)

(2698,) (540,) (540,) (540,)


In [50]:
ensamble_log = logreg.fit(pd.DataFrame([dev_predictions_logreg, dev_predictions_nb, dev_predictions_svm]), df.Category)

ValueError: Found arrays with inconsistent numbers of samples: [   3 2698]

In [ ]:
crossvalidation(model, train_features, real_category)

**Submit test set to Kaggle**

In [ ]:
test_predictions_nb = Predict(nb, tfidf_vec, df, df_test, 'cv')
train_predictions_nb = Predict(nb, tfidf_vec, df, df, 'cv')

In [115]:
create_submission("submission_7.csv", test_predictions)

In [ ]:
test_predictions_logreg = Predict(LogisticRegression, tfidf_vec, df, df_test, 'cv')

In [ ]:
#this one needs tuning...
test_predictions_KNN = Predict(KNeighborsClassifier, tfidf_vec, df, df_test, 'cv', num_neighbors=3)

In [ ]:
# tried gamma = 0.001, 0.003, 0.005 and 0.007. 0.005 has the best result
test_predictions_svm = Predict(svm, tfidf_vec, df, df_test, 'cv')

In [ ]:
create_submission("submission_8.csv", test_predictions_logreg)

### apply voting system between three models: logreg, nb, and svm 

In [ ]:
voted_results = []
for triple in zip(test_predictions_logreg, test_predictions_nb, test_predictions_svm):
    voted_results.append(vote_winner(triple))
voted_results = pd.Series(voted_results)

In [ ]:
print(df.Category.shape, voted_results.shape)

In [272]:
create_submission("submission_9.csv", test_predictions_nb)

In [ ]:
test_predictions_nb = Predict(nb, tfidf_vec, df, df_test, 'cv')

## Analyzing Confusion Matrix

In [ ]:
train_predictions_svm = Predict(svm_model, tfidf_vec, df_train, df_train, 'cv')
train_predictions_svm.shape

In [ ]:
# Credit to Marti H. And John S. for the following function:
def plot_confusion_matrix(cm, title, target_names, cmap=plt.cm.coolwarm):
    plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
class_labels = np.sort(df_train.Category.unique())

In [ ]:
nb_cm = confusion_matrix(df_train.Category, train_predictions_svm)
plot_confusion_matrix(nb_cm, "SVM Confusion Matrix", class_labels)

In [ ]:
pd.crosstab(df_train.Category, train_predictions_svm, 
            rownames=['True'], colnames=['Predicted'], 
            margins=True)

In [ ]:
df_cat1 = df.ix[:,0:2]

In [ ]:
series_temp = df_cat1['Category'] 
len(series_temp)

In [ ]:
series_temp = df_cat1['Category'] 
for n,i in enumerate(list(series_temp)):
    if i != 1: 
        series_temp[n] = 0
series_temp

In [ ]:
len(series_temp)